In [15]:
import numpy as np

def print_tableau(tableau, iteration=None):
    if iteration is not None:
        print(f"\n--- Iteration {iteration} ---")
    print("Current Tableau:")
    # Print a rounded copy of the tableau for easier reading
    print(np.round(tableau, 3))
    print("-" * 50)

def tabular_simplex(objective_coeffs, constraint_matrix, rhs_values, senses, problem_type='max', verbose=True):
    num_constraints, num_original_vars = constraint_matrix.shape

    # Transform constraints if needed (e.g., converting '>=' constraints into '<=' constraints)
    transformed_constraint_matrix, transformed_rhs_values = transform_constraints(constraint_matrix, rhs_values, senses)
    tableau = setup_tableau(objective_coeffs, transformed_constraint_matrix, transformed_rhs_values, senses, problem_type)

    if verbose:
        print("\nInitial Tableau:")
        print_tableau(tableau)
        print("Explanation: We reach the solution when all coefficients in the objective row (first row) "
              "are nonnegative. This means that no further improvements (increase in the choice variable) can "
              "be achieved.")

    # Initial infeasibility check (e.g., checking artificial variables)
    status = check_infeasibility(tableau, num_original_vars, senses, num_constraints)
    if status == 'infeasible':
        if verbose:
            print("The problem is infeasible at the initial tableau.")
        return status, None, None

    iteration = 0
    while True:
        iteration += 1

        if verbose:
            print_tableau(tableau, iteration)
        
        # Check the optimality condition: If all coefficients in the objective row (except RHS) are nonnegative,
        # then the current solution is optimal.
        if np.all(tableau[0, :-1] >= 0):
            if verbose:
                print("All coefficients in the objective row are now nonnegative.")
                print("Explanation: No further improvement is possible so the current solution is optimal.")
            optimal_solution, optimal_objective_value = extract_solution(tableau, num_original_vars, num_constraints, problem_type)
            if np.any(np.dot(transformed_constraint_matrix, optimal_solution) > transformed_rhs_values + 1e-6):
                if verbose:
                    print("After checking, there is a violation in the constraints (infeasible basic variable)!")
                return 'infeasible', None, None
            if verbose:
                print("Optimal solution reached!")
                print("Solution:", np.round(optimal_solution, 3))
                print("Objective value:", round(optimal_objective_value, 3))
            return 'optimal', optimal_solution, optimal_objective_value

        # Select entering variable: the variable with the most negative coefficient in the objective row.
        entering_col_index = select_entering_variable(tableau)
        if verbose:
            print(f"Entering variable (most negative coefficient) is at column index: {entering_col_index}")
            print(f"Coefficient value for entering variable: {tableau[0, entering_col_index]:.3f}")

        # Calculate ratios to determine the leaving variable.
        ratios = calculate_ratios(tableau, entering_col_index)
        if verbose:
            print("Calculating ratios for the pivot operation (RHS divided by pivot column coefficient):")
            for idx, ratio in enumerate(ratios, start=1):
                if tableau[idx, entering_col_index] > 0:
                    print(f"Row {idx}: RHS = {tableau[idx, -1]:.3f}, Coefficient = {tableau[idx, entering_col_index]:.3f}, Ratio = {ratio:.3f}")
                else:
                    print(f"Row {idx}: Coefficient = {tableau[idx, entering_col_index]:.3f} (Not eligible for pivot, ratio = inf)")
                    
        # Select leaving variable based on the minimum ratio rule.
        leaving_row = select_leaving_variable(tableau, entering_col_index)
        if leaving_row is None:
            if verbose:
                print("No valid leaving variable found (all ratios are infinite). The problem is unbounded!")
            return 'unbounded', None, None

        if verbose:
            print(f"Leaving variable is in row: {leaving_row}")

        # Perform the pivot operation.
        tableau = pivot(tableau, entering_col_index, leaving_row)
        if verbose:
            print("After pivot operation, the tableau is updated as follows:")
            print_tableau(tableau, iteration)

def check_infeasibility(tableau, num_original_vars, senses, num_constraints):
    # This check looks for artificial basic variables with nonzero values.
    num_slack_vars = senses.count('<=')
    artificial_vars_start = num_original_vars + num_slack_vars

    for i in range(num_constraints):
        basic_variable_cols = np.where(tableau[i+1, :num_original_vars + num_slack_vars + num_constraints] == 1)[0]
        if len(basic_variable_cols) > 0:
            basic_var = basic_variable_cols[0]
            if basic_var >= artificial_vars_start and tableau[i+1, -1] != 0:
                return 'infeasible'
    return None

def select_entering_variable(tableau):
    # Identify the most negative coefficient in the objective row (excluding the RHS).
    return np.argmin(tableau[0, :-1])

def calculate_ratios(tableau, entering_col_index):
    ratios = []
    # Compute ratios for each constraint (row 1 and onward)
    for i in range(1, tableau.shape[0]):
        if tableau[i, entering_col_index] > 0:
            ratios.append(tableau[i, -1] / tableau[i, entering_col_index])
        else:
            ratios.append(np.inf)
    return np.array(ratios)

def select_leaving_variable(tableau, entering_col_index):
    ratios = calculate_ratios(tableau, entering_col_index)
    if np.all(ratios == np.inf):
        return None
    # Add one because the first row is the objective function
    return np.argmin(ratios) + 1

def pivot(tableau, entering_col_index, leaving_row):
    pivot_element = tableau[leaving_row, entering_col_index]
    tableau[leaving_row, :] = tableau[leaving_row, :] / pivot_element
    # Adjust all other rows to zero out the entering variable column.
    for i in range(tableau.shape[0]):
        if i != leaving_row:
            factor = tableau[i, entering_col_index]
            tableau[i, :] -= factor * tableau[leaving_row, :]
    return tableau

def setup_tableau(objective_coeffs, constraint_matrix, rhs_values, senses, problem_type='max'):
    num_constraints, num_original_vars = constraint_matrix.shape

    if problem_type == 'min':
        # Convert minimization to maximization by negating the objective coefficients.
        objective_coeffs = -objective_coeffs

    num_slack_vars = senses.count('<=')
    num_surplus_vars = senses.count('>=')
    num_artificial_vars = senses.count('=') + num_surplus_vars
    num_total_vars = num_original_vars + num_slack_vars + num_artificial_vars

    tableau = np.zeros((num_constraints + 1, num_total_vars + 1))
    tableau[0, :num_original_vars] = -objective_coeffs  # Negate objective coefficients here

    slack_surplus_index = num_original_vars
    artificial_index = num_original_vars + num_slack_vars + num_surplus_vars
    for i in range(num_constraints):
        tableau[i + 1, :num_original_vars] = constraint_matrix[i, :]
        tableau[i + 1, -1] = rhs_values[i]

        if senses[i] == '<=':
            tableau[i + 1, slack_surplus_index] = 1
            slack_surplus_index += 1
        elif senses[i] == '>=' or senses[i] == '=':
            # For '>=' or '=' constraints, add an artificial variable to help find a basic feasible solution.
            tableau[i + 1, artificial_index - num_surplus_vars] = 1
            artificial_index += 1
            if senses[i] == '>=':
                # Also add a surplus variable (which will have a negative sign)
                tableau[i + 1, slack_surplus_index] = -1
                slack_surplus_index += 1
    return tableau

def extract_solution(tableau, num_original_vars, num_constraints, problem_type='max'):
    optimal_solution = np.zeros(num_original_vars)
    # Look through each original variable column to see if it's a basic variable.
    for i in range(num_original_vars):
        column = tableau[:, i]
        if np.sum(np.abs(column)) == 1 and np.count_nonzero(column == 1) == 1:
            basic_variable_row = np.where(column == 1)[0][0]
            optimal_solution[i] = tableau[basic_variable_row, -1]

    optimal_objective_value = tableau[0, -1]
    if problem_type == 'min':
        optimal_objective_value = -optimal_objective_value
    return optimal_solution, optimal_objective_value

def transform_constraints(constraint_matrix, rhs_values, senses):
    transformed_constraint_matrix = constraint_matrix.copy()
    transformed_rhs_values = rhs_values.copy()
    for i, sense in enumerate(senses):
        if sense == '>=':
            # Multiply the constraint by -1 to convert '>=' into '<='.
            transformed_constraint_matrix[i, :] = -constraint_matrix[i, :]
            transformed_rhs_values[i] = -rhs_values[i]
    return transformed_constraint_matrix, transformed_rhs_values


In [16]:
# Example usage:
if __name__ == "__main__":
    # Define a sample linear programming problem.
    # Example: Maximize Z = 3x + 5y
    # Subject to:
    #   2x + y <= 10
    #   x + 3y <= 12
    #   x, y >= 0
    objective_coeffs = np.array([3, 5])
    constraint_matrix = np.array([[2, 1],
                                  [1, 3]])
    rhs_values = np.array([10, 12])
    senses = ['<=', '<=']  # Both constraints are of the form <=

    status, solution, objective_value = tabular_simplex(objective_coeffs, constraint_matrix, rhs_values, senses, problem_type='max', verbose=True)
    print("\nFinal Result:")
    print("Status:", status)
    if solution is not None:
        print("Solution:", np.round(solution, 3))
    if objective_value is not None:
        print("Objective Value:", round(objective_value, 3))


Initial Tableau:
Current Tableau:
[[-3. -5.  0.  0.  0.]
 [ 2.  1.  1.  0. 10.]
 [ 1.  3.  0.  1. 12.]]
--------------------------------------------------
Explanation: We reach the solution when all coefficients in the objective row (first row) are nonnegative. This means that no further improvements (increase in the choice variable) can be achieved.

--- Iteration 1 ---
Current Tableau:
[[-3. -5.  0.  0.  0.]
 [ 2.  1.  1.  0. 10.]
 [ 1.  3.  0.  1. 12.]]
--------------------------------------------------
Entering variable (most negative coefficient) is at column index: 1
Coefficient value for entering variable: -5.000
Calculating ratios for the pivot operation (RHS divided by pivot column coefficient):
Row 1: RHS = 10.000, Coefficient = 1.000, Ratio = 10.000
Row 2: RHS = 12.000, Coefficient = 3.000, Ratio = 4.000
Leaving variable is in row: 2
After pivot operation, the tableau is updated as follows:

--- Iteration 1 ---
Current Tableau:
[[-1.333  0.     0.     1.667 20.   ]
 [ 1.667

In [17]:
from webapp.example_problems import example_problems

for problem_name, problem_data in example_problems.items():
    objective_coeffs = np.array(problem_data["objective_coeffs"])
    constraint_matrix = np.array(problem_data["constraint_matrix"])
    rhs_values = np.array(problem_data["rhs_values"])
    senses = problem_data["senses"]
    problem_type = problem_data["problem_type"]

    status, solution, objective_value = tabular_simplex(objective_coeffs, constraint_matrix, rhs_values, senses, problem_type)

    print(f"\n{problem_name}:")
    print("Status:", status)
    if solution is not None:
        print("Solution:", solution)
    if objective_value is not None:
        print("Objective Value:", objective_value)


Initial Tableau:
Current Tableau:
[[-3. -5.  0.  0.  0.  0.]
 [ 1.  0.  1.  0.  0.  4.]
 [ 0.  2.  0.  1.  0. 12.]
 [ 3.  2.  0.  0.  1. 18.]]
--------------------------------------------------
Explanation: We reach the solution when all coefficients in the objective row (first row) are nonnegative. This means that no further improvements (increase in the choice variable) can be achieved.

--- Iteration 1 ---
Current Tableau:
[[-3. -5.  0.  0.  0.  0.]
 [ 1.  0.  1.  0.  0.  4.]
 [ 0.  2.  0.  1.  0. 12.]
 [ 3.  2.  0.  0.  1. 18.]]
--------------------------------------------------
Entering variable (most negative coefficient) is at column index: 1
Coefficient value for entering variable: -5.000
Calculating ratios for the pivot operation (RHS divided by pivot column coefficient):
Row 1: Coefficient = 0.000 (Not eligible for pivot, ratio = inf)
Row 2: RHS = 12.000, Coefficient = 2.000, Ratio = 6.000
Row 3: RHS = 18.000, Coefficient = 2.000, Ratio = 9.000
Leaving variable is in row: 2
